In [0]:

%sql
-- Volume para zona raw (arquivos JSON)
CREATE VOLUME IF NOT EXISTS lakehouse.raw_public.yfinance

In [0]:
%python
%pip install yfinance

In [0]:
dbutils.library.restartPython()

In [0]:
import yfinance as yf
import pandas as pd
from datetime import datetime, UTC

# UC Volume path (Raw Zone)
RAW_BASE_PATH = "/Volumes/lakehouse/raw_public/yfinance/commodities/latest_prices"

# Ensures the directory exists (Databricks)
dbutils.fs.mkdirs(RAW_BASE_PATH)

def get_commodities_df() -> pd.DataFrame:
    """
    Returns the latest quotes (1 minute) for Gold, Oil, and Silver via Yahoo Finance.
    """
    symbols = ["GC=F", "CL=F", "SI=F"]  # Gold, Oil, Silver
    dfs = []

    for sym in symbols:
        try:
            # 1-day history with 1-minute interval, gets the last one
            ultimo_df = yf.Ticker(sym).history(period="1d", interval="1m")[["Close"]].tail(1)
            if ultimo_df.empty:
                continue

            ultimo_df = ultimo_df.rename(columns={"Close": "preco"})
            ultimo_df["ativo"] = sym
            ultimo_df["moeda"] = "USD"
            ultimo_df["horario_coleta"] = datetime.now(UTC).isoformat()
            dfs.append(ultimo_df[["ativo", "preco", "moeda", "horario_coleta"]])

        except Exception as e:
            print(f"⚠️ Error fetching {sym}: {e}")

    if not dfs:
        raise ValueError("No quotes returned by Yahoo Finance.")

    df = pd.concat(dfs, ignore_index=True)
    df["source_system"] = "yfinance"
    df["source_endpoint"] = "https://finance.yahoo.com"
    df["ingestion_ts_utc"] = datetime.now(UTC).isoformat()

    return df


# ===== Collection =====
df = get_commodities_df()

# ===== Save path =====
file_name = f"{RAW_BASE_PATH}/yfinance_commodities_{datetime.now(UTC).strftime('%Y%m%d_%H%M%S')}.json"

# ===== Simple JSON write =====
df.to_json(file_name, orient="records", lines=True, force_ascii=False)

print(f"JSON saved at: {file_name}")